In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import csv

print("✅ Libraries loaded.")


/Users/IDEAPAD/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Libraries loaded.


In [2]:
features = ['gr','dt', 'dt_nct', 'rhob_combined', 'res_deep', 'sphi', 'hp', 'ob']
target_col = 'ppp'
depth_col = 'depth'

files = ['QAZIAN -1X.CSV', 'MISSA KESWAL-01.CSV', 'MISSA KESWAL-03.CSV']
print("✅ Config set.")


✅ Config set.


In [3]:
def detect_delimiter(file_path):
    with open(file_path, 'r') as f:
        sample = f.readline()
        sniffer = csv.Sniffer()
        return sniffer.sniff(sample).delimiter

data_frames = []
for f in files:
    try:
        delim = detect_delimiter(f)
        df = pd.read_csv(f, delimiter=delim, engine='python')
        df.columns = df.columns.str.strip().str.lower()
        df.replace(-999.25, np.nan, inplace=True)
        df = df[features + [target_col, depth_col]]
        df.dropna(inplace=True)
        data_frames.append(df)
        print(f"✅ Loaded and cleaned {f}")
    except Exception as e:
        print(f"❌ Failed to load {f}: {e}")

df = pd.concat(data_frames, ignore_index=True)
print("✅ All files merged.")


✅ Loaded and cleaned QAZIAN -1X.CSV
✅ Loaded and cleaned MISSA KESWAL-01.CSV
✅ Loaded and cleaned MISSA KESWAL-03.CSV
✅ All files merged.


In [4]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
y_train = train_df[target_col].values
X_test = scaler.transform(test_df[features])
y_test = test_df[target_col].values

X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print("✅ Data split and scaled.")


✅ Data split and scaled.


In [5]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)
print("✅ Callbacks set.")


✅ Callbacks set.


In [7]:
cnn = Sequential([
    Input(shape=(X_train_cnn.shape[1], 1)),
    Conv1D(64, kernel_size=2, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=2, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1)
])
cnn.compile(optimizer='adam', loss='mse')
cnn.fit(X_train_cnn, y_train, validation_split=0.2, epochs=100, batch_size=32,
        callbacks=[early_stop, reduce_lr], verbose=1)
y_pred_cnn = cnn.predict(X_test_cnn).flatten()
print("✅ CNN model trained.")


Epoch 1/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7220710.0000 - val_loss: 1717481.5000 - learning_rate: 0.0010
Epoch 2/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 456294.5938 - val_loss: 315250.1562 - learning_rate: 0.0010
Epoch 3/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 326405.0938 - val_loss: 182332.9844 - learning_rate: 0.0010
Epoch 4/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 324701.1250 - val_loss: 164984.9844 - learning_rate: 0.0010
Epoch 5/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 292771.1875 - val_loss: 165510.8906 - learning_rate: 0.0010
Epoch 6/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 287459.0938 - val_loss: 155191.8906 - learning_rate: 0.0010
Epoch 7/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 277504.5000 - val_loss: 155841.2500 - learning_rate: 0.0010
Epoch 8/100
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 281191.4062 - val_loss: 154694.7812 - learning_rate: 0.0010
Epoch 9/100
259/259 ━━━━━━━━━━

In [8]:
# Save the model to your preferred directory
cnn.save('/Users/IDEAPAD/Documents/Research work/Geoscience project/Well Data/PPCNN/cnn_model.h5')
print("✅ CNN model saved successfully.")


✅ CNN model saved successfully.


In [9]:
# dfnn = Sequential([
#     Input(shape=(X_train.shape[1],)),
#     Dense(256, activation='relu'),
#     Dropout(0.3),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(1)
# ])
# dfnn.compile(optimizer='adam', loss='mse')
# dfnn.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=32,
#          callbacks=[early_stop, reduce_lr], verbose=1)
# y_pred_dfnn = dfnn.predict(X_test).flatten()
# print("✅ DFNN model trained.")

# 🔧 Tunable parameters (can be changed for performance tuning)
input_dim = X_train.shape[1]
hidden_units_1 = 128   # Number of neurons in first hidden layer
hidden_units_2 = 64    # Number of neurons in second hidden layer
dropout_rate_1 = 0.3   # Dropout after first hidden layer
dropout_rate_2 = 0.3   # Dropout after second hidden layer
epochs = 100           # Number of training iterations
batch_size = 32        # Batch size for training

# Define DFNN model
dfnn = Sequential([
    Dense(hidden_units_1, activation='relu', input_shape=(input_dim,)),
    BatchNormalization(),
    Dropout(dropout_rate_1),
    
    Dense(hidden_units_2, activation='relu'),
    BatchNormalization(),
    Dropout(dropout_rate_2),
    
    Dense(1)  # Output layer for regression
])

# Compile model
dfnn.compile(optimizer='adam', loss='mse')

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)

# Train the model
dfnn.fit(X_train, y_train,
         validation_split=0.3,
         epochs=epochs,
         batch_size=batch_size,
         callbacks=[early_stop, reduce_lr],
         verbose=1)

# Make predictions on test set
y_pred_dfnn = dfnn.predict(X_test).flatten()

print("✅ DFNN training and prediction complete.")


Epoch 1/100


/Users/IDEAPAD/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


227/227 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 8833658.0000 - val_loss: 8577747.0000 - learning_rate: 0.0010
Epoch 2/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - loss: 8814681.0000 - val_loss: 8395779.0000 - learning_rate: 0.0010
Epoch 3/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 8578485.0000 - val_loss: 8099934.5000 - learning_rate: 0.0010
Epoch 4/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - loss: 8302709.0000 - val_loss: 7757114.0000 - learning_rate: 0.0010
Epoch 5/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7785275.0000 - val_loss: 7329316.0000 - learning_rate: 0.0010
Epoch 6/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 7419655.5000 - val_loss: 6931929.5000 - learning_rate: 0.0010
Epoch 7/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 6967361.0000 - val_loss: 6394048.0000 - learning_rate: 0.0010
Epoch 8/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 6438979.0000 - val_loss: 5904133.5000 - learning_rate: 0.0010
Epoch 9/100
227/

In [12]:
# Save the model to your preferred directory
dfnn.save('/Users/IDEAPAD/Documents/Research work/Geoscience project/Well Data/PPCNN/dfnn_model.h5')
print("✅ DFNN model saved successfully.")


✅ DFNN model saved successfully.


In [11]:
def evaluate_model(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    rel_rmse = (rmse / (max(y_true) - min(y_true))) * 100
    print(f"📊 {name}:")
    print(f"   R²     : {r2:.4f}")
    print(f"   MSE    : {mse:.2f}")
    print(f"   RMSE   : {rmse:.2f}")
    print(f"   MAE    : {mae:.2f}")
    print(f"   Rel RMSE: {rel_rmse:.2f}% of PPP range\n")

evaluate_model("CNN", y_test, y_pred_cnn)
evaluate_model("DFNN", y_test, y_pred_dfnn)


📊 CNN:
   R²     : 0.8687
   MSE    : 108739.40
   RMSE   : 329.76
   MAE    : 243.41
   Rel RMSE: 6.36% of PPP range

📊 DFNN:
   R²     : 0.9004
   MSE    : 82459.23
   RMSE   : 287.16
   MAE    : 200.43
   Rel RMSE: 5.54% of PPP range

